In [17]:
!pip install torch
!pip install botorch

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [18]:
pip install --upgrade botorch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [28]:
#defining everything for dummy testing

param_bounds = {
    'c1': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1.0, 'name': 'W', 'unit': 'dimensionless', 'type': 'hardening'}, 
    'c2': {'lowerBound': 0, 'upperBound': 2, 'exponent': 1000.0, 'name': 'K', 'unit': 'MPa', 'type': 'yielding'}, 
    'c3': {'lowerBound': 0, 'upperBound': 1, 'exponent': 0.1, 'name': 'e0', 'unit': 'dimensionless', 'type': 'hardening'}, 
    'c4': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1.0, 'name': 'n', 'unit': 'dimensionless', 'type': 'hardening'}, 
    'c5': {'lowerBound': 0, 'upperBound': 2, 'exponent': 1000.0, 'name': 'sigma_y', 'unit': 'MPa', 'type': 'yielding'}, 
    'c6': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1000.0, 'name': 'sigma_sat', 'unit': 'MPa', 'type': 'hardening'}, 
    'c7': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1000.0, 'name': 'b', 'unit': 'dimensionless', 'type': 'hardening'}
}


geometries = ['NDBR50', 'NDBR6', 'CHD6']

def calculate_loss(simulated_curve, target_curve):
    # This is a placeholder. Replace this with your actual loss function.
    return np.sum((simulated_curve - target_curve)**2)


def generate_dummy_target_curves():
    geometries = ['NDBR50', 'NDBR6', 'CHD6']
    targetCurves = {geometry: {'force': np.random.rand(100), 'displacement': np.linspace(0, 2, 100)} for geometry in geometries}
    return targetCurves

def generate_dummy_combined_data():
    geometries = ['NDBR50', 'NDBR6', 'CHD6']
    params = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7']
    combined_data = {
        tuple((param, np.random.rand()) for param in params): 
        {geometry: {'force': np.random.rand(100), 'displacement': np.linspace(0, 2, 100)} for geometry in geometries}
        for _ in range(100)
    }
    return combined_data

In [33]:
import torch
import numpy as np
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
from botorch.acquisition.multi_objective import qExpectedHypervolumeImprovement
from botorch.optim import optimize_acqf
from botorch.utils.multi_objective.box_decompositions import NondominatedPartitioning

# Define your custom loss function


# Generate dummy data
combined_interpolated_param_to_geom_FD_Curves_smooth = generate_dummy_combined_data()
targetCurves = generate_dummy_target_curves()

# Convert your data to the required format
param_data = []
objective_data = []
for param_tuple, geom_dict in combined_interpolated_param_to_geom_FD_Curves_smooth.items():
    param_data.append([value for param, value in param_tuple])
    objective_data.append([calculate_loss(geom_dict[geometry]["force"], targetCurves[geometry]["force"]) for geometry in geometries])
your_param_data = torch.tensor(param_data).float()
your_objective_data = torch.tensor(objective_data).float()

# Normalize the parameters
normalized_param_data = (your_param_data - your_param_data.min(dim=0).values) / (your_param_data.max(dim=0).values - your_param_data.min(dim=0).values)

# Train a GP model
gp = SingleTaskGP(normalized_param_data, your_objective_data)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll)

# Define the bounds of the search space
lower_bounds = torch.tensor([param_bounds[param]['lowerBound'] for param in param_bounds.keys()]).float()
upper_bounds = torch.tensor([param_bounds[param]['upperBound'] for param in param_bounds.keys()]).float()
bounds = torch.stack([lower_bounds, upper_bounds])

# Define the reference point as slightly worse than the worst observed values
ref_point = your_objective_data.max(dim=0).values + 0.1

# Define the partitioning of the output space
partitioning = botorch.utils.multi_objective.box_decompositions.non_dominated.NondominatedPartitioning(ref_point=ref_point, Y=your_objective_data)

# Define the acquisition function
acq_func = qExpectedHypervolumeImprovement(model=gp, partitioning=partitioning, ref_point=torch.zeros(your_objective_data.size(-1)).tolist())

# Optimize the acquisition function to find the next set of parameters to evaluate
candidates, _ = optimize_acqf(
    acq_function=acq_func,
    bounds=bounds,
    q=10,  # change this to the number of candidates you want to generate
    num_restarts=5,
    raw_samples=20,
)

# Denormalize the candidates
next_params = candidates * (your_param_data.max(dim=0).values - your_param_data.min(dim=0).values) + your_param_data.min(dim=0).values

# Convert the tensor to a list of dictionaries
next_param_dicts = [{param: value.item() for param, value in zip(param_bounds.keys(), next_param)} for next_param in next_params]

# Select the non-dominated solutions
pareto_mask = is_non_dominated(torch.stack([gp.posterior(next_param.unsqueeze(0)).mean for next_param in next_params]))
pareto_solutions = [next_param_dicts[i] for i in range(len(next_param_dicts)) if pareto_mask[i]]

print(pareto_solutions)


C:\Users\zopev1\AppData\Roaming\Python\Python39\site-packages\botorch\models\gpytorch.py:129: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(_get_single_precision_warning(X.dtype), UserWarning)
C:\Users\zopev1\AppData\Roaming\Python\Python39\site-packages\botorch\models\utils\assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


[{'c1': 0.8792486190795898, 'c2': 1.992463231086731, 'c3': 0.6484872102737427, 'c4': 0.7131639122962952, 'c5': 1.8831543922424316, 'c6': 0.496869832277298, 'c7': 0.9574712514877319}, {'c1': 0.2613418400287628, 'c2': 1.50616455078125, 'c3': 0.8306484222412109, 'c4': 0.08781611174345016, 'c5': 1.9354491233825684, 'c6': 0.6911563277244568, 'c7': 0.7134778499603271}, {'c1': 0.5536680817604065, 'c2': 0.1731215864419937, 'c3': 0.20179393887519836, 'c4': 0.4600904881954193, 'c5': 1.0931661128997803, 'c6': 0.188849538564682, 'c7': 0.4555857479572296}, {'c1': 0.9301822781562805, 'c2': 0.08139479160308838, 'c3': 0.9568307995796204, 'c4': 0.29582470655441284, 'c5': 1.9144587516784668, 'c6': 0.7132220268249512, 'c7': 0.6579915881156921}, {'c1': 0.030092626810073853, 'c2': 0.1482355296611786, 'c3': 0.8750647902488708, 'c4': 0.7462082505226135, 'c5': 0.9299249053001404, 'c6': 0.18878304958343506, 'c7': 0.6040541529655457}, {'c1': 0.9769315123558044, 'c2': 1.2655372619628906, 'c3': 0.6432329416275024

C:\Users\zopev1\AppData\Roaming\Python\Python39\site-packages\botorch\optim\initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(
